In [1]:
%load_ext autoreload
%autoreload 2

import joblib as jl

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.svm import SVR

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.metrics import mean_squared_error

import functions as f
import classes as c

In [ ]:
train_df = pd.read_csv("datasets/train.csv", na_values="np.NaN").set_index("Id")
test_df = pd.read_csv("datasets/test.csv", na_values=np.NaN).set_index("Id")
corr_matrix = train_df.corr()
# train_df[pd.isnull(train_df)] = np.NaN

train_df.loc[:, "LotAreaSqrt"] = np.sqrt(train_df["LotArea"].values)
train_df.loc[:, "GarageAreaSqrt"] = np.sqrt(train_df["GarageArea"].values)

col_of_interest = ["LotArea", "LotAreaSqrt", "GarageArea", "GarageAreaSqrt", "LotFrontage"]
train_df = train_df[col_of_interest].copy()
train_df.dropna(axis=0, inplace=True)

X = train_df.drop("LotFrontage", axis=1).copy()
y = train_df["LotFrontage"].copy()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6],
                     'C': [1, 10, 100, 1000, 10000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000, 10000]}]

In [6]:
clf = GridSearchCV(SVR(), tuned_parameters, cv=5, scoring="neg_mean_squared_error")

In [ ]:
clf.fit(X_train, y_train)
print("Best parameters set found on development set:")
print(clf.best_params_)

In [ ]:
model = clf.best_estimator_

In [ ]:
y_train_pr = model.predict(X_train)
y_test_pr = model.predict(X_test)
mse_train = mean_squared_error(y_train, y_train_pr)
mse_test = mean_squared_error(y_test, y_test_pr)

In [ ]:
np.sqrt(mse_train), np.sqrt(mse_test)